# Obtain a List of S3 URLs for a GES DISC Collection Using the CMR API
### Authors: Chris Battisto, Alexis Hunzinger
### Date Authored: 1-31-22
### Date Updated: 2-1-23

### Timing

Exercise: 15 minutes

### Overview

This notebook demonstrates how to obtain a list of S3 URLs for desired cloud-hosted GES DISC granules using the [Commmon Metadata Repository (CMR) API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).

### Prerequisites

This notebook was written using Python 3.8, and requires these libraries and files: 
- requests

Identify your data collection of interest and acquire its shortname or concept ID.
- These can be found on the collection's Dataset Landing Page on the GES DISC website. For example, this is the Dataset Landing Page for Hourly MERRA-2 SLV: https://disc.gsfc.nasa.gov/datasets/M2T1NXSLV_5.12.4/summary  


## Import Libraries

In [1]:
import requests

## Create a Function for CMR Catalog Requests

In [2]:
def cmr_request(params):
    response = requests.get(url,
                        params=params,
                        headers={
                            'Accept': 'application/json',
                        }
                       )
    return response

## Search CMR Catalogs and Obtain S3 URLs

### Check that the CMR catalog can be accessed

If "200, CMR is accessible" is returned, the catalog can be accessed!

In [3]:
url = 'https://cmr.earthdata.nasa.gov/search/collections'

# Create our request for finding cloud-hosted granules, and check that we can access CMR
response = cmr_request({
                        'cloud_hosted': 'True',
                        'has_granules': 'True'
                        })

if response.status_code == 200:
    print(str(response.status_code) + ", CMR is accessible")
else:
    print(str(response.status_code) + ", CMR is not accessible, check for outages")

200, CMR is accessible


### Search CMR with your desired data collection's shortname or concept ID and desired date range

Using the collection's shortname or concept ID, we can obtain individual granules by querying https://cmr.earthdata.nasa.gov/search/granules. By querying a JSON response of the granules that we want, we can obtain each granule's S3 URL. 

Here, we will parse out an S3 URL for the AQUA AIRS IR + MW Level 2 CLIMCAPS dataset. 
- **Shortname**: SNDRAQIML2CCPRET
- **Concept ID**: C1693440798-GES_DISC

Our desried date range is 00:00:00 to 06:00:00 on January 1, 2016. 

In [4]:
url = 'https://cmr.earthdata.nasa.gov/search/granules'

shortname = 'SNDRAQIML2CCPRET'
concept_id = 'C1693440798-GES_DISC'

start_time = '2016-01-01T00:00:00Z'
end_time = '2016-01-01T06:00:00Z'

# OPTION 1: Using shortname
response = cmr_request({
                        'shortname': shortname,
                        'temporal': start_time+','+end_time,
                        })

# OPTION 2: Using concept ID
response = cmr_request({
                        'concept_id': concept_id,
                        'temporal': start_time+','+end_time,
                        })

# Pretty print the JSON response, where we can see which entry contains the S3 link:
#pprint.pprint(response.json()['feed']['entry'][0]['links'])

#print(response.json()['feed']['entry'])
print(response.headers['CMR-Hits'])
granules = response.json()['feed']['entry']
#print(granules)

61


Now, we can parse out that link, and assign it to a variable:

In [6]:
#climcaps_s3_link = response.json()['feed']['entry'][0]['links'][1]['href']
#print(climcaps_s3_link)

#[print('Granule',i['links']) for i in granules]
# for i in granules:
#     print(i['links'])

#'title': 'This link provides direct download access via S3 to the granule'
#'href' is they key for the S3 links

test = granules[0]['links']

In [7]:
for granule in granules:
    print(f'{granule['links']}')

SyntaxError: f-string: unmatched '[' (4199281754.py, line 2)